In [18]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import os
import calendar

from plotly.offline import init_notebook_mode, iplot, plot, download_plotlyjs

import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

#### Merge 12 months of sales data into one dataframe
# df = pd.read_csv('csv_files/Sales_April_2019.csv')
# df.head()

# we find all the csv files in the folder
files = [file for file in os.listdir('csv_files') if not file.startswith('.')]

# create an empty dataframe
all_months_data = pd.DataFrame()

# Loop through all the files and append the data into the dataframe
for file in files:
    df = pd.read_csv('csv_files/' + file)
    all_months_data = pd.concat([all_months_data, df])
    
all_months_data.head()
# This will create a new csv file with all the merged data
# all_months_data.to_csv('csv_files/all_data.csv', index=False)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


Augment file to add columns


In [58]:
all_months_data = pd.read_csv('csv_files/all_data.csv')
# all_months_data.head()

# check to see if rows have NaN
# all_months_data[all_months_data.isna().any(axis=1)]

# Remove NaN rows
all_months_data = all_months_data.dropna()
all_months_data = all_months_data[all_months_data['Order Date'].notna()]
# This will exclude all rows with 'Or' in the 'Order Date' column
all_months_data = all_months_data[all_months_data['Order Date'].str[0:2] != 'Or']

# add month column
all_months_data['Month'] = all_months_data['Order Date'].str[0:2]
all_months_data['Month'] = pd.to_numeric(all_months_data['Month'], errors='coerce')

# Add city column
all_months_data['City'] = all_months_data['Purchase Address'].apply(lambda x: x.split(',')[1] + ',' + x.split(',')[2].split(' ')[1])

# Make some cleaning before converting to numeric
all_months_data['Quantity Ordered'] = pd.to_numeric(all_months_data['Quantity Ordered'])
all_months_data['Price Each'] = pd.to_numeric(all_months_data['Price Each'])

# Format 'Month' column with leading zeros
# all_months_data['Month'] = all_months_data['Month'].apply(lambda x: f"{int(x):02}" if not pd.isna(x) else x)

# Convert numeric month to month name
all_months_data['Month'] = all_months_data['Month'].apply(lambda x: calendar.month_abbr[int(x)] if not pd.isna(x) else x)

# Get a total of sales by month
# all_months_data = all_months_data.groupby('Month').sum()

# Convert 'Month' column to categorical with specified order
# Define the order of months
# month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# month_order = all_months_data['Month'].value_counts().index
month_order = all_months_data['Month'].value_counts(dropna=False).index



all_months_data['Month'] = pd.Categorical(all_months_data['Month'], categories=month_order, ordered=True)

# Sort the DataFrame based on the 'Month' column order
all_months_data = all_months_data.sort_values('Month')

# Get sales values 
all_months_data['Sales'] = all_months_data['Quantity Ordered'] * all_months_data['Price Each']


all_months_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,City,Sales
36347,301336,Vareebadd Phone,1,400.00,12/18/19 02:05,"176 Center St, Los Angeles, CA 90001",Dec,"Los Angeles,CA",400.00
34327,299405,Bose SoundSport Headphones,1,99.99,12/10/19 19:05,"207 12th St, Portland, ME 04101",Dec,"Portland,ME",99.99
34326,299404,USB-C Charging Cable,1,11.95,12/21/19 10:18,"878 Lincoln St, Boston, MA 02215",Dec,"Boston,MA",11.95
34325,299403,Lightning Charging Cable,1,14.95,12/19/19 09:59,"484 North St, San Francisco, CA 94016",Dec,"San Francisco,CA",14.95
34324,299402,USB-C Charging Cable,1,11.95,12/18/19 20:19,"507 14th St, San Francisco, CA 94016",Dec,"San Francisco,CA",11.95


What was the best month for sales? How much was earned?

In [59]:
# Get total sales by month
sales_by_month = all_months_data.groupby('Month')['Sales'].sum()
# all_months_data = all_months_data.groupby('Month').sum()

# Let's plot this	
x = sales_by_month.index
y = sales_by_month.values

fig = go.Figure()

fig.add_trace(go.Bar(
    x=x,
    y=y,
    marker_color='purple',
    opacity=0.7
))

fig.update_layout(
    title='Total Sales by Month',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Total Sales'),
)

fig.show()

# all_months_data

C:\Users\USER\AppData\Local\Temp\ipykernel_11960\3355022744.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Which City recorded the best sales

In [60]:
# Get total sales by city
sales_by_city = all_months_data.groupby('City')['Sales'].sum()
# all_months_data = all_months_data.groupby('Month').sum()

# Let's plot this	
x = sales_by_city.index
y = sales_by_city.values

fig = go.Figure()

fig.add_trace(go.Bar(
    x=x,
    y=y,
    marker_color='blue',
    opacity=0.7
))

fig.update_layout(
    title='Total Sales by City',
    xaxis=dict(title='City'),
    yaxis=dict(title='Total Sales'),
)

fig.show()

What time should we display advertisements to maximize sales?

In [69]:
# What time should we display advertisement to maximize likelihood of customer buying product?

# Peak activity time for sales
# Assuming you have a DataFrame called 'all_months_data' with a column named 'OrderDate' containing the sales order dates

# Convert 'Order Date' column to datetime format
all_months_data['Order Time'] = pd.to_datetime(all_months_data['Order Date'])

# Extract the hour from the 'Order Time' column
all_months_data['Hour'] = all_months_data['Order Time'].dt.hour

# Group by hour and count the number of sales
sales_by_hour = all_months_data.groupby('Hour').size()

# Find the hour with the highest number of sales
peak_activity_time = sales_by_hour.idxmax()

# Print the peak activity time
print("Peak activity time for sales is at hour:", peak_activity_time)
# all_months_data

# let's plot this
x = sales_by_hour.index.astype(str) 
y = sales_by_hour.values

fig = go.Figure()

fig.add_trace(go.Bar(
    x=x,
    y=y,
    marker_color='red',
    opacity=0.7
))

fig.update_layout(
    title='Sales by Hour',
    xaxis=dict(title='Hour'),
    yaxis=dict(title='Number of Sales'),
)

fig.show()


C:\Users\USER\AppData\Local\Temp\ipykernel_11960\308771182.py:7: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23],
      dtype='int32', name='Hour')
Hour
0      3910
1      2350
2      1243
3       831
4       854
5      1321
6      2482
7      4011
8      6256
9      8748
10    10944
11    12411
12    12587
13    12129
14    10984
15    10175
16    10384
17    10899
18    12280
19    12905
20    12228
21    10921
22     8822
23     6275
dtype: int64
Peak activity time for sales is at hour: 19


What products are often sold together?

In [80]:
from itertools import combinations
from collections import Counter

# Find out which products are often sold together
# from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import association_rules

# # Convert the transaction data into a one-hot encoded matrix
# one_hot_encoded_data = pd.get_dummies(transaction_data)

# # Generate frequent itemsets using the apriori algorithm
# frequent_itemsets = apriori(one_hot_encoded_data, min_support=0.05, use_colnames=True)

# # Generate association rules from the frequent itemsets
# association_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# # Sort the association rules by confidence in descending order
# sorted_rules = association_rules.sort_values(by=['confidence'], ascending=False)

# # Print the top 10 association rules
# print(sorted_rules.head(10))

# Get duplicated orders based on Order ID
dp_orders = all_months_data[all_months_data['Order ID'].duplicated(keep=False)]

# Now we can add a new column named 'Group' to the DataFrame that combines the 'Order ID' and 'Product' columns
dp_orders['Group'] = dp_orders.groupby('Order ID')['Product'].transform(lambda x: ', '.join(x))

# Next, we can drop the duplicate rows based on the 'Order ID' column
# dp_orders= dp_orders.drop_duplicates(subset=['Order ID'], inplace=True)
dp_orders = dp_orders[['Order ID','Group']].drop_duplicates()

# 
count = Counter()
most_sold_products_grouped = dp_orders.groupby('Group')['Group'].count().sort_values(ascending=False).head(10)

for row in dp_orders['Group']:
    row_list = row.split(',')
    # count.update(Counter(combinations(row_list, 2)))
    for length in range(2, len(row_list) + 1):
        count.update(Counter(combinations(row_list, length)))
 
# Print the most sold grouped products 
for key, val in count.most_common(50):
    print(key,val)

# dp_orders.head(20)



('Google Phone', ' USB-C Charging Cable') 520
('iPhone', ' Lightning Charging Cable') 486
('Lightning Charging Cable', ' iPhone') 462
('USB-C Charging Cable', ' Google Phone') 421
('Wired Headphones', ' iPhone') 227
('iPhone', ' Wired Headphones') 227
('Google Phone', ' Wired Headphones') 205
('Wired Headphones', ' Google Phone') 201
('Vareebadd Phone', ' USB-C Charging Cable') 194
('iPhone', ' Apple Airpods Headphones') 178
('Apple Airpods Headphones', ' iPhone') 177
('USB-C Charging Cable', ' Vareebadd Phone') 148
('Google Phone', ' Bose SoundSport Headphones') 116
('Bose SoundSport Headphones', ' Google Phone') 98
('Wired Headphones', ' USB-C Charging Cable') 92
('Vareebadd Phone', ' Wired Headphones') 79
('Wired Headphones', ' Lightning Charging Cable') 69
('Wired Headphones', ' Vareebadd Phone') 67
('Apple Airpods Headphones', ' Lightning Charging Cable') 65
(' Lightning Charging Cable', ' iPhone') 64
('Wired Headphones', ' Apple Airpods Headphones') 57
('USB-C Charging Cable', ' 